In [1]:
import os
import requests
import numpy as np
import pandas as pd

from api_keys import fmp_api_key as api_key
from API import API

api = API(api_key)
X = api.getSamples("TSLA")

display(X)
print(X.dtypes)
print(len(X))

,symbol,date,currentRatio,quickRatio,grossProfitMargin,operatingProfitMargin,returnOnAssets,returnOnEquity,returnOnCapitalEmployed,debtEquityRatio,...,growthGrossProfitRatio,growthOperatingExpenses,growthEBITDA,growthOperatingIncome,growthNetIncome,growthEPS,symbol,date,marketCapitalization,enterpriseValue
0,TSLA,2016-12-31,1.074273,0.667986,0.228461,-0.095332,-0.029779,-0.142000,-0.044328,3.603263,...,0.000927,0.381960,-2.195635,-0.068779,-0.240529,-0.324675,TSLA,2016-12-31,3.614385e+10,3.987907e+10
1,TSLA,2017-12-31,0.856131,0.505989,0.189007,-0.138798,-0.068448,-0.462895,-0.105289,5.527365,...,-0.172694,0.700599,-1.254705,1.445659,1.906148,1.527778,TSLA,2017-12-31,5.732243e+10,6.426946e+10
2,TSLA,2018-12-31,0.831285,0.463829,0.188340,-0.011781,-0.032821,-0.198262,-0.050880,4.871174,...,-0.003528,0.114225,-17.651361,-0.845082,-0.502350,-0.516484,TSLA,2018-12-31,5.265301e+10,6.093876e+10
3,TSLA,2019-12-31,1.134621,0.711728,0.165555,0.003255,-0.025125,-0.130251,-0.028128,4.055908,...,-0.120982,-0.071216,0.370818,-1.316406,-0.116886,-0.139860,TSLA,2019-12-31,1.134234e+11,1.205744e+11
4,TSLA,2020-12-31,1.875140,1.492841,0.210236,0.063229,0.013232,0.031046,0.030449,1.308121,...,0.269889,0.162196,0.857942,23.925000,-2.000000,-2.269589,TSLA,2020-12-31,7.403635e+11,7.327185e+11


symbol                             object
date                               object
currentRatio                      float64
quickRatio                        float64
grossProfitMargin                 float64
operatingProfitMargin             float64
returnOnAssets                    float64
returnOnEquity                    float64
returnOnCapitalEmployed           float64
debtEquityRatio                   float64
priceToBookRatio                  float64
priceToSalesRatio                 float64
priceEarningsRatio                float64
priceEarningsToGrowthRatio        float64
priceToOperatingCashFlowsRatio    float64
symbol                             object
date                               object
growthRevenue                     float64
growthCostOfRevenue               float64
growthGrossProfit                 float64
growthGrossProfitRatio            float64
growthOperatingExpenses           float64
growthEBITDA                      float64
growthOperatingIncome             